# Intro to Torch.ONNX—End-to-end AlexNet from PyTorch to Caffe2

## Exporting model from PyTorch to ONNX.

In this tutorial, we describe how to use ONNX to convert a model defined in PyTorch into the ONNX format.

ONNX is natively supported in PyTorch. However, ONNX exporter is **not yet** part of the [PyTorch repository](http://pytorch.org/docs/master/onnx.html) as what they mentioned.

For working with this tutorial, you will need to install [ONNX](https://github.com/onnx/onnx). You can get binary builds of ONNX with `conda install onnx`.

_NOTE: ONNX is under active development so for the best support consider building PyTorch master branch which can be installed by following [the instructions here](https://github.com/pytorch/pytorch#from-source) or install PyTorch preview using `conda install pytorch-nightly cuda92 -c pytorch`._

We will export a pretrained AlexNet as defined in `torchvision` into ONNX. It runs a single round of inference and then saves the resulting traced model to `alexnet.onnx` file.

### Invoking exporter

Pretty much it's a matter of replacing `my_model(input)` with `torch.onnx.export(my_model, input, "my_model.onnx")` in your script.

#### Limitations

The ONNX exporter is a trace-based exporter, which means that it operates by executing your model once, and exporting the operators which were actually run during this run. This means that if your model is dynamic, e.g., changes behavior depending on input data, the export won’t be accurate.

Similarly, a trace is might be valid only for a specific input size (which is one reason why we require explicit inputs on tracing). Most of the operators export size-agnostic versions and should work on different batch sizes or input sizes. We recommend examining the model trace and making sure the traced operators look reasonable.

In [3]:
import torch
import torchvision

### Trying it out on AlexNet

If you already have your model built, it's just a few lines:

In [4]:
# Standard ImageNet input - 3 channels, 224x224,
# values don't matter as we care about network structure.
# But they can also be real inputs.
dummy_input = torch.rand(10, 3, 224, 224, device='cuda')
# Obtain your model, it can be also constructed in your script explicitly
model = torchvision.models.alexnet(pretrained=True).cuda()

Providing input and output names sets the display names for values within the model's graph. Setting these does not change the semantics of the graph; it is only for readability.

The inputs to the network consist of the flat list of inputs (i.e. the values you would pass to the forward() method) followed by the flat list of parameters. You can partially specify names, i.e. provide a list here shorter than the number of inputs to the model, and we will only set that subset of names, starting from the beginning.

In [5]:
input_names = ["actual_input_1"] + ["learned_%d" % i for i in range(16)]
output_names = ["output1"]

Invoke export:

In [6]:
torch.onnx.export(model, dummy_input, "alexnet.onnx", verbose=True, input_names=input_names,
                  output_names=output_names)

graph(%actual_input_1 : Float(10, 3, 224, 224)
      %learned_0 : Float(64, 3, 11, 11)
      %learned_1 : Float(64)
      %learned_2 : Float(192, 64, 5, 5)
      %learned_3 : Float(192)
      %learned_4 : Float(384, 192, 3, 3)
      %learned_5 : Float(384)
      %learned_6 : Float(256, 384, 3, 3)
      %learned_7 : Float(256)
      %learned_8 : Float(256, 256, 3, 3)
      %learned_9 : Float(256)
      %learned_10 : Float(4096, 9216)
      %learned_11 : Float(4096)
      %learned_12 : Float(4096, 4096)
      %learned_13 : Float(4096)
      %learned_14 : Float(1000, 4096)
      %learned_15 : Float(1000)) {
  %17 : Float(10, 64, 55, 55) = onnx::Conv[dilations=[1, 1], group=1, kernel_shape=[11, 11], pads=[2, 2, 2, 2], strides=[4, 4]](%actual_input_1, %learned_0, %learned_1), scope: AlexNet/Sequential[features]/Conv2d[0]
  %18 : Float(10, 64, 55, 55) = onnx::Relu(%17), scope: AlexNet/Sequential[features]/ReLU[1]
  %19 : Float(10, 64, 27, 27) = onnx::MaxPool[kernel_shape=[3, 3], pads=[0, 0, 

**That's it!**

The resulting ``alexnet.onnx`` is a **binary protobuf file** which contains both
the network structure and parameters of the model you exported
(in this case, AlexNet).  The keyword argument ``verbose=True`` causes the
exporter to print out a human-readable representation of the network:

### Inspecting model

You can also use ONNX tooling to check the validity of the resulting model or inspect the details.

You can verify the protobuf using the ONNX library.

#### Install ONNX

First, we need to install ONNX library:

You can install onnx with conda:

`conda install -c conda-forge onnx`

In [5]:
import onnx

ModuleNotFoundError: No module named 'onnx'

**Fix the previous error**

_WARNING: Don't install `onnx` from conda-forge channel. Run this command instead:_

`conda install onnx`

In [7]:
import onnx

Load the ONNX model:

In [8]:
model = onnx.load("alexnet.onnx")

Check that the IR is well formed:

In [9]:
onnx.checker.check_model(model)

Print a human readable representation of the graph:

In [10]:
print(onnx.helper.printable_graph(model.graph))

graph torch-jit-export (
  %actual_input_1[FLOAT, 10x3x224x224]
) initializers (
  %learned_0[FLOAT, 64x3x11x11]
  %learned_1[FLOAT, 64]
  %learned_2[FLOAT, 192x64x5x5]
  %learned_3[FLOAT, 192]
  %learned_4[FLOAT, 384x192x3x3]
  %learned_5[FLOAT, 384]
  %learned_6[FLOAT, 256x384x3x3]
  %learned_7[FLOAT, 256]
  %learned_8[FLOAT, 256x256x3x3]
  %learned_9[FLOAT, 256]
  %learned_10[FLOAT, 4096x9216]
  %learned_11[FLOAT, 4096]
  %learned_12[FLOAT, 4096x4096]
  %learned_13[FLOAT, 4096]
  %learned_14[FLOAT, 1000x4096]
  %learned_15[FLOAT, 1000]
) {
  %17 = Conv[dilations = [1, 1], group = 1, kernel_shape = [11, 11], pads = [2, 2, 2, 2], strides = [4, 4]](%actual_input_1, %learned_0, %learned_1)
  %18 = Relu(%17)
  %19 = MaxPool[kernel_shape = [3, 3], pads = [0, 0, 0, 0], strides = [2, 2]](%18)
  %20 = Conv[dilations = [1, 1], group = 1, kernel_shape = [5, 5], pads = [2, 2, 2, 2], strides = [1, 1]](%19, %learned_2, %learned_3)
  %21 = Relu(%20)
  %22 = MaxPool[kernel_shape = [3, 3], pads = [0

Notice that all parameters are listed as graph's inputs but they also have stored values initialized in `model.graph.initializers`.

### Caffe2

To run the following code with [Caffe2](https://caffe2.ai/), you will need to install Caffe2: If you don’t have one already, Please [follow the install instructions](https://caffe2.ai/docs/getting-started.html).

_**NOTE: actually Caffe2 is already installed when you install PyTorch.**_ The following describe why the confusion:

I referenced the following GitHub issues because of the discrepancies in current PyTorch docs and Caffe2 docs. The PyTorch docs lacks Caffe2 information, not clear enough or outdated. This confusing state has spawned out a large chunk of GitHub issues, for examples:
- [Why it is so difficult to install caffe2??? (Jan 2018)](https://github.com/caffe2/caffe2/issues/1811)
- [How to intall and use Caffe2 from this Pytorch branch?](https://github.com/pytorch/pytorch/issues/7029)
- [Caffe2 Installation inside Pytorch](https://github.com/pytorch/pytorch/issues/12646)

Once these are installed, you can use the backend for Caffe2:

In [9]:
# ...continuing from above
import caffe2.python.onnx.backend as backend
import numpy as np

ModuleNotFoundError: No module named 'past'

Fix the previous error by installing the `future` package:

`conda install future`

Ref: https://github.com/caffe2/caffe2/issues/712#issuecomment-305962653

In [12]:
# ...continuing from above
import caffe2.python.onnx.backend as backend
import numpy as np

In [13]:
rep = backend.prepare(model, device="CUDA:0") # or "CPU"

CUDA operators do not support 64-bit doubles, please use arr.astype(np.float32) or np.int32 for ints. Blob: actual_input_1 type: float64


#### Get Predictions by Running the Model

The results come back as a multidimensional array of probabilities. Essentially each row is a percentage chance that the object matches something that the neural net recognizes. For example, when you run the flower it should give you over 95% rating that the flower is a daisy.

In [14]:
# For the Caffe2 backend:
#     rep.predict_net is the Caffe2 protobuf for the network
#     rep.workspace is the Caffe2 workspace for the network
#       (see the class caffe2.python.onnx.backend.Workspace)
outputs = rep.run(np.random.randn(10, 3, 224, 224).astype(np.float32))

In [15]:
# To run networks with more than one input, pass a tuple
# rather than a single numpy ndarray.
print(outputs[0])

[[-0.16684037 -1.7584792  -1.6565322  ... -1.5301019  -0.83980227
   0.8916631 ]
 [ 0.38660896 -0.897966   -1.2219435  ... -1.168266   -1.0750638
   0.9224857 ]
 [ 0.0033838  -1.3918537  -1.6014808  ... -1.1375068  -1.1307131
   1.1512557 ]
 ...
 [-0.04392347 -1.2045223  -1.4013005  ... -0.9982521  -1.0994978
   1.2750273 ]
 [ 0.18175897 -1.5210615  -1.7670833  ... -1.3994161  -0.93560624
   0.86104625]
 [ 0.4133255  -1.2434499  -1.2564434  ... -1.3632581  -0.90603685
   1.039825  ]]


In [16]:
outputs[0].shape # 10 inputs and predictions (ImageNet 1000 classes) for each input

(10, 1000)

### ========================== END OF TUTORIAL ==========================

---

## More on ONNX

#### LIMITATIONS

- The ONNX exporter is a trace-based exporter, which means that it operates by executing your model once, and exporting the operators which were actually run during this run. This means that if your model is dynamic, e.g., changes behavior depending on input data, the export won’t be accurate. Similarly, a trace is likely to be valid only for a specific input size (which is one reason why we require explicit inputs on tracing.) We recommend examining the model trace and making sure the traced operators look reasonable.
- PyTorch and Caffe2 often have implementations of operators with some numeric differences. Depending on model structure, these differences may be negligible, but they can also cause major divergences in behavior (especially on untrained models.) In a future release, we plan to allow Caffe2 to call directly to Torch implementations of operators, to help you smooth over these differences when precision is important, and to also document these differences.

#### SUPPORTED OPERATORS

The following operators are supported:

- add (nonzero alpha not supported)
- sub (nonzero alpha not supported)
- mul
- div
- cat
- mm
- addmm
- neg
- sqrt
- tanh
- sigmoid
- mean
- sum
- prod
- t
- expand (only when used before a broadcasting ONNX operator; e.g., add)
- transpose
- view
- split
- squeeze
- ...and many more

The operator set above is sufficient to export the following models:

- AlexNet
- DCGAN
- DenseNet
- Inception (warning: this model is highly sensitive to changes in operator implementation)
- ResNet
- SuperResolution
- VGG
- word_language_model

Adding export support for operators is an _advance_ usage. To achieve this, developers need to touch the source code of PyTorch. Please follow the [instructions](https://github.com/pytorch/pytorch#from-source) for installing PyTorch from source. If the wanted operator is standardized in ONNX, it should be easy to add support for exporting such operator (adding a symbolic function for the operator). To confirm whether the operator is standardized or not, please check the [ONNX operator list](https://github.com/onnx/onnx/blob/master/docs/Operators.md).

If the operator is an ATen operator, which means you can find the declaration of the function in `torch/csrc/autograd/generated/VariableType.h` (available in generated code in PyTorch install dir), you should add the symbolic function in `torch/onnx/symbolic.py` and follow the instructions listed as below:

- Define the symbolic function in [torch/onnx/symbolic.py](https://github.com/pytorch/pytorch/blob/master/torch/onnx/symbolic.py). Make sure the function has the same name as the ATen operator/function defined in `VariableType.h`.
- The first parameter is always the exported ONNX graph. Parameter names must EXACTLY match the names in `VariableType.h`, because dispatch is done with keyword arguments.
- Parameter ordering does NOT necessarily match what is in `VariableType.h`, tensors (inputs) are always first, then non-tensor arguments.
- In the symbolic function, if the operator is already standardized in ONNX, we only need to create a node to represent the ONNX operator in the graph.
- If the input argument is a tensor, but ONNX asks for a scalar, we have to explicitly do the conversion. The helper function `_scalar` can convert a scalar tensor into a python scalar, and `_if_scalar_type_as` can turn a Python scalar into a PyTorch tensor.

If the operator is a non-ATen operator, the symbolic function has to be added in the corresponding PyTorch Function class. Please read the following instructions:

- Create a symbolic function named `symbolic` in the corresponding Function class.
- The first parameter is always the exported ONNX graph.
- Parameter names except the first must EXACTLY match the names in `forward`.
- The output tuple size must match the outputs of `forward`.
- In the symbolic function, if the operator is already standardized in ONNX, we just need to create a node to represent the ONNX operator in the graph.

Symbolic functions should be implemented in Python. All of these functions interact with Python methods which are implemented via C++-Python bindings, but intuitively the interface they provide looks like this:

In [ ]:
def operator/symbolic(g, *inputs):
  """
  Modifies Graph (e.g., using "op"), adding the ONNX operations representing
  this PyTorch function, and returning a Value or tuple of Values specifying the
  ONNX outputs whose values correspond to the original PyTorch return values
  of the autograd Function (or None if an output is not supported by ONNX).

  Arguments:
    g (Graph): graph to write the ONNX representation into
    inputs (Value...): list of values representing the variables which contain
        the inputs for this function
  """

class Value(object):
  """Represents an intermediate tensor value computed in ONNX."""
  def type(self):
    """Returns the Type of the value."""

class Type(object):
  def sizes(self):
    """Returns a tuple of ints representing the shape of a tensor this describes."""

class Graph(object):
  def op(self, opname, *inputs, **attrs):
    """
    Create an ONNX operator 'opname', taking 'args' as inputs
    and attributes 'kwargs' and add it as a node to the current graph,
    returning the value representing the single output of this
    operator (see the `outputs` keyword argument for multi-return
    nodes).

    The set of operators and the inputs/attributes they take
    is documented at https://github.com/onnx/onnx/blob/master/docs/Operators.md

    Arguments:
        opname (string): The ONNX operator name, e.g., `Abs` or `Add`.
        args (Value...): The inputs to the operator; usually provided
            as arguments to the `symbolic` definition.
        kwargs: The attributes of the ONNX operator, with keys named
            according to the following convention: `alpha_f` indicates
            the `alpha` attribute with type `f`.  The valid type specifiers are
            `f` (float), `i` (int), `s` (string) or `t` (Tensor).  An attribute
            specified with type float accepts either a single float, or a
            list of floats (e.g., you would say `dims_i` for a `dims` attribute
            that takes a list of integers).
        outputs (int, optional):  The number of outputs this operator returns;
            by default an operator is assumed to return a single output.
            If `outputs` is greater than one, this functions returns a tuple
            of output `Value`, representing each output of the ONNX operator
            in positional.
    """

The ONNX graph C++ definition is in `torch/csrc/jit/ir.h`.

Here is an example of handling missing symbolic function for `elu` operator. We try to export the model and see the error message as below:

```sh
UserWarning: ONNX export failed on elu because torch.onnx.symbolic.elu does not exist
RuntimeError: ONNX export failed: Couldn't export operator elu
```

The export fails because PyTorch does not support exporting `elu` operator. We find `virtual` `Tensor` `elu(const` `Tensor` `&` `input`, `Scalar` `alpha`, `bool` `inplace)` `const` `override;` in `VariableType.h`. This means `elu` is an ATen operator. We check the [ONNX operator list](http://https//github.com/onnx/onnx/blob/master/docs/Operators.md), and confirm that `Elu` is standardized in ONNX. We add the following lines to `symbolic.py`:

In [ ]:
def elu(g, input, alpha, inplace=False):
    return g.op("Elu", input, alpha_f=_scalar(alpha))

Now PyTorch is able to export `elu` operator.

There are more examples in [symbolic.py](https://github.com/pytorch/pytorch/blob/master/torch/onnx/symbolic.py), [tensor.py](https://github.com/pytorch/pytorch/blob/99037d627da68cdf53d3d0315deceddfadf03bba/torch/autograd/_functions/tensor.py#L24), [padding.py](https://github.com/pytorch/pytorch/blob/99037d627da68cdf53d3d0315deceddfadf03bba/torch/nn/_functions/padding.py#L8).

The interface for specifying operator definitions is experimental; adventurous users should note that the APIs will probably change in a future interface.

#### FUNCTIONS

`torch.onnx.export(*args, **kwargs)`. [Source](https://pytorch.org/docs/master/_modules/torch/onnx.html#export)

---

## ONNX Issues

I experienced the following issues related to ONNX or the old ONNX-Caffe2 integration along the way in this task:
- [RESOLVED] Error: "ONNX checker Bad node spec" and GitHub issues referenced:
  - https://github.com/onnx/onnx/issues/932
- [RESOLVED] Error: "onnx_cpp2py_export.cpython-36m-x86_64-linux-gnu.so: undefined symbol: ZNK6google8protobuf7Message11GetTypeNameB5cxx11Ev" and GitHub issues referenced:
  - [SOLUTION] https://github.com/onnx/onnx/issues/745
  - https://github.com/onnx/onnx/issues/1516
  - https://github.com/onnx/onnx-caffe2/issues/19

#### Protocol Buffer Required by ONNX

I experienced the following issues related to protobuf along the way in this task:
- I am using Ubuntu 16.04 LTS. It came with a default protobuf library, `protobuf.so.9`. This is causing some serious conflicts wth the one ONNX trying to install and link to. I have not tested this with Ubuntu 18.04 LTS yet.
- [RESOLVED] Error: "... SIGABRT ..."
 - https://github.com/BVLC/caffe/issues/5711
- [RESOLVED] Error: "Segmentation Fault on ONNX"
  - https://github.com/onnx/onnx/issues/875
- [RESOLVED] Error: "ValueError: The graph does not have an ir_version set properly."
  - https://github.com/onnx/onnx/issues/95#issuecomment-335515684